# Weather Prediction App

In [1]:

# Import necessary libraries
import ipywidgets as widgets
from IPython.display import display, Markdown
from meteostat import Point, Daily
import torch
import pandas as pd
from sklearn.preprocessing import QuantileTransformer
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

# Title of the app
display(Markdown("# Weather Prediction App"))

# Input date widget
date_picker = widgets.DatePicker(
    description="Pick a Date",
    disabled=False,
    value=datetime.today()
)

# Button to trigger predictions
predict_button = widgets.Button(
    description="Get Predictions",
    button_style="primary"
)

# Output area to display results
output_area = widgets.Output()

# Function to fetch weather data and make predictions
def fetch_weather_predictions(date_input):
    try:
        date_object = datetime.strptime(str(date_input), "%Y-%m-%d")
    except ValueError:
        output_area.clear_output()
        with output_area:
            display(Markdown("**Error:** Invalid date format. Please use YYYY-MM-DD."))
        return
    
    previous_day = date_object - timedelta(days=1)
    date_one_month_ago = date_object - relativedelta(months=1)
    end = datetime(previous_day.year, previous_day.month, previous_day.day)
    start = datetime(date_one_month_ago.year, date_one_month_ago.month, date_one_month_ago.day)

    # Location (New York City)
    location = Point(40.7128, -74.0060)
    
    # Fetch data from Meteostat API
    input_data = Daily(location, start, end)
    input_data = input_data.fetch()
    input_data = pd.DataFrame(input_data)
    
    # Preprocessing
    input_data.drop(['wpgt', 'tsun', 'wdir'], axis=1, inplace=True)
    input_data.fillna(0, inplace=True)
    input_data.replace([float('inf'), -float('inf')], 0, inplace=True)

    # Load the model
    model = torch.jit.load('model_scripted.pt')
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Scaling the input data
    scaler = QuantileTransformer(output_distribution='uniform')
    input_data_scaled = scaler.fit_transform(input_data)
    input_data_scaled = input_data_scaled[-30:]
    input_data_scaled = torch.tensor(input_data_scaled).view(1, 30, 7).float()
    input_data_scaled = input_data_scaled.to(device)

    # Make predictions
    with torch.no_grad():
        predictions = model(input_data_scaled)
    predictions_rescaled = predictions.cpu().numpy()
    predictions_rescaled = scaler.inverse_transform(predictions.cpu().numpy().reshape(-1, input_data.shape[1]))

    result = dict(zip(input_data.columns, predictions_rescaled[0]))
    legend = {
        'tavg': "Avg Temp in °C",
        'tmin': 'Min Temp in °C',
        'tmax': 'Max Temp in °C',
        'prcp': 'Precipitation in mm',
        'snow': 'Snow in mm',
        'wspd': 'Windspeed in km/h',
        'pres': 'Atmospheric Pressure in hPa'
    }

    return result, legend

# Callback function to display results
def on_predict_button_clicked(b):
    output_area.clear_output()
    date_input = date_picker.value
    if not date_input:
        with output_area:
            display(Markdown("**Error:** Please select a valid date."))
        return
    
    result, legend = fetch_weather_predictions(date_input)
    with output_area:
        display(Markdown(f"## Weather Predictions for {date_input}"))
        for key, value in result.items():
            display(Markdown(f"- **{legend[key]}:** {value:.2f}"))

# Attach the callback to the button
predict_button.on_click(on_predict_button_clicked)

# Display widgets
display(date_picker, predict_button, output_area)


# Weather Prediction App

DatePicker(value=datetime.datetime(2024, 12, 5, 16, 20, 33, 843405), description='Pick a Date', step=1)

Button(button_style='primary', description='Get Predictions', style=ButtonStyle())

Output()